In [1]:
import pandas as pd
from sqlalchemy import create_engine


### Store CSV into DataFrame

In [2]:
from secrets import username, password


In [69]:
csv_file = "../kroger/kroger_milk_prices.csv"
kroger_data_df = pd.read_csv(csv_file)
kroger_data_df.head()


,Unnamed: 0,productId,categories,brand,description,size,price.regular,price.promo,Date,Store_Id,image
0,0,1111040101,['Dairy'],Kroger,Kroger® Vitamin D Whole Milk,1 gal,2.99,0.0,02/08/2021,3400312,https://www.kroger.com/product/images/xlarge/f...
1,1,1111041600,['Dairy'],Kroger,Kroger® 2% Reduced Fat Milk,1/2 gal,1.99,0.0,02/08/2021,3400312,https://www.kroger.com/product/images/xlarge/f...
2,2,1111041660,['Dairy'],Kroger,Kroger® 1% Lowfat Milk,1 gal,2.99,0.0,02/08/2021,3400312,https://www.kroger.com/product/images/xlarge/f...
3,3,1111040601,['Dairy'],Kroger,Kroger® Vitamin D Whole Milk,1/2 gal,1.99,0.0,02/08/2021,3400312,https://www.kroger.com/product/images/xlarge/f...
4,4,1111042315,['Dairy'],Kroger,Kroger® Fat Free Skim Milk,1 gal,2.99,0.0,02/08/2021,3400312,https://www.kroger.com/product/images/xlarge/f...


In [4]:
milk_df = kroger_data_df
milk_df = milk_df.drop(columns=['Unnamed: 0', 'description' , 'price.regular', 'price.promo', 'Date'])
features = 'none'
milk_df['features'] = features
milk_df = milk_df.rename(columns={"productId": "product_id", "brand": "Brand" , "categories" : "category" , 'Store_Id' : 'Store_Number'})
milk_df['Type_ID'] = 1
milk_df.head()

,product_id,category,Brand,size,Store_Number,image,features,Type_ID
0,1111040101,['Dairy'],Kroger,1 gal,3400312,https://www.kroger.com/product/images/xlarge/f...,none,1
1,1111041600,['Dairy'],Kroger,1/2 gal,3400312,https://www.kroger.com/product/images/xlarge/f...,none,1
2,1111041660,['Dairy'],Kroger,1 gal,3400312,https://www.kroger.com/product/images/xlarge/f...,none,1
3,1111040601,['Dairy'],Kroger,1/2 gal,3400312,https://www.kroger.com/product/images/xlarge/f...,none,1
4,1111042315,['Dairy'],Kroger,1 gal,3400312,https://www.kroger.com/product/images/xlarge/f...,none,1


In [70]:
store_df = pd.DataFrame()
store_id = 3400312 , 1111111
store_df["Store_Number"] = store_id
zipcode = 77007 , 77007
store_df['Store_Zipcode'] = zipcode
name = "Kroger" , "HEB"
store_df['Store_Name'] = name
store_df.head()

,Store_Number,Store_Zipcode,Store_Name
0,3400312,77007,Kroger
1,1111111,77007,HEB


In [71]:
temp_milk =pd.DataFrame()
store_id = 3400312 , 1111111
temp_milk["Store_Number"] = store_id
zipcode = 77007 , 77007
temp_milk['Store_Zipcode'] = zipcode
name = "Kroger" , "HEB"
temp_milk['Store_Name'] = name
temp_milk.head()

,Store_Number,Store_Zipcode,Store_Name
0,3400312,77007,Kroger
1,1111111,77007,HEB


In [72]:
temp_milk = pd.concat([temp_milk, store_df], axis=0).drop_duplicates().reset_index(drop=True)
temp_milk.head()

,Store_Number,Store_Zipcode,Store_Name
0,3400312,77007,Kroger
1,1111111,77007,HEB


In [6]:
milk_type_df = pd.DataFrame()
milk_type = ['1%' , "2%", 'whole', 'skim', 'almond', 'other']
milk_type_df['Type'] = milk_type
milk_type_df.head()

,Type
0,1%
1,2%
2,whole
3,skim
4,almond


In [7]:
price_history_df = kroger_data_df[['Date', 'price.regular', 'price.promo', 'productId']].copy()
price_history_df =price_history_df.rename(columns={'Date':'date', 'price.regular':'price', 'price.promo':'saleprice', 'productId': 'product_id'})
price_history_df.head()

,date,price,saleprice,product_id
0,02/08/2021,2.99,0.0,1111040101
1,02/08/2021,1.99,0.0,1111041600
2,02/08/2021,2.99,0.0,1111041660
3,02/08/2021,1.99,0.0,1111040601
4,02/08/2021,2.99,0.0,1111042315


### Connect to local database

In [8]:
rds_connection_string = f"{username}:{password}@localhost:5432/milk_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

### Check for tables

In [9]:
engine.table_names()

['Store', 'Milk', 'Milk_Type', 'Price_History']

### Use pandas to load csv converted DataFrame into database

In [10]:
store_df.to_sql(name='Store', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the customer_name table
* NOTE: can also check using pgAdmin

In [11]:
pd.read_sql_query('select * from "Store"', con=engine).head()

,id,Store_Number,Store_Name,Store_Zipcode
0,1,3400312,Kroger,77007
1,2,1111111,HEB,77007


### Use pandas to load json converted DataFrame into database

In [12]:
milk_type_df.to_sql(name='Milk_Type', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the customer_location table

In [13]:
pd.read_sql_query('select * from "Milk_Type"', con=engine).head()

,id,Type
0,1,1%
1,2,2%
2,3,whole
3,4,skim
4,5,almond


In [14]:
stores2_df = pd.read_sql_query('select * from "Store"', con=engine).head()
milk_df = milk_df.merge(stores2_df, on = 'Store_Number')
milk_df.head()

,product_id,category,Brand,size,Store_Number,image,features,Type_ID,id,Store_Name,Store_Zipcode
0,1111040101,['Dairy'],Kroger,1 gal,3400312,https://www.kroger.com/product/images/xlarge/f...,none,1,1,Kroger,77007
1,1111041600,['Dairy'],Kroger,1/2 gal,3400312,https://www.kroger.com/product/images/xlarge/f...,none,1,1,Kroger,77007
2,1111041660,['Dairy'],Kroger,1 gal,3400312,https://www.kroger.com/product/images/xlarge/f...,none,1,1,Kroger,77007
3,1111040601,['Dairy'],Kroger,1/2 gal,3400312,https://www.kroger.com/product/images/xlarge/f...,none,1,1,Kroger,77007
4,1111042315,['Dairy'],Kroger,1 gal,3400312,https://www.kroger.com/product/images/xlarge/f...,none,1,1,Kroger,77007


In [15]:
milk_df = milk_df.drop(columns = ['Store_Name' , 'Store_Zipcode', 'Store_Number'])
milk_df.head()

,product_id,category,Brand,size,image,features,Type_ID,id
0,1111040101,['Dairy'],Kroger,1 gal,https://www.kroger.com/product/images/xlarge/f...,none,1,1
1,1111041600,['Dairy'],Kroger,1/2 gal,https://www.kroger.com/product/images/xlarge/f...,none,1,1
2,1111041660,['Dairy'],Kroger,1 gal,https://www.kroger.com/product/images/xlarge/f...,none,1,1
3,1111040601,['Dairy'],Kroger,1/2 gal,https://www.kroger.com/product/images/xlarge/f...,none,1,1
4,1111042315,['Dairy'],Kroger,1 gal,https://www.kroger.com/product/images/xlarge/f...,none,1,1


In [16]:
milk_df = milk_df.rename(columns ={'id' : 'Store_ID'})
milk_df.head()

,product_id,category,Brand,size,image,features,Type_ID,Store_ID
0,1111040101,['Dairy'],Kroger,1 gal,https://www.kroger.com/product/images/xlarge/f...,none,1,1
1,1111041600,['Dairy'],Kroger,1/2 gal,https://www.kroger.com/product/images/xlarge/f...,none,1,1
2,1111041660,['Dairy'],Kroger,1 gal,https://www.kroger.com/product/images/xlarge/f...,none,1,1
3,1111040601,['Dairy'],Kroger,1/2 gal,https://www.kroger.com/product/images/xlarge/f...,none,1,1
4,1111042315,['Dairy'],Kroger,1 gal,https://www.kroger.com/product/images/xlarge/f...,none,1,1


In [17]:
milk_df.to_sql(name='Milk', con=engine, if_exists='append', index=False)

In [18]:
pd.read_sql_query('select * from "Milk"', con=engine).head()

,id,product_id,Brand,features,size,category,image,Store_ID,Type_ID
0,1,1.111040e+09,Kroger,none,1 gal,['Dairy'],https://www.kroger.com/product/images/xlarge/f...,1,1
1,2,1.111042e+09,Kroger,none,1/2 gal,['Dairy'],https://www.kroger.com/product/images/xlarge/f...,1,1
2,3,1.111042e+09,Kroger,none,1 gal,['Dairy'],https://www.kroger.com/product/images/xlarge/f...,1,1
3,4,1.111041e+09,Kroger,none,1/2 gal,['Dairy'],https://www.kroger.com/product/images/xlarge/f...,1,1
4,5,1.111042e+09,Kroger,none,1 gal,['Dairy'],https://www.kroger.com/product/images/xlarge/f...,1,1


In [19]:
milk2_df = pd.read_sql_query('select * from "Milk"', con=engine).head()
milk2_df = milk2_df.drop(columns = ['Brand' , 'size', 'image', 'Store_ID', 'Type_ID', 'category', 'features'])
milk2_df.head()
price_history_df = price_history_df.merge(milk2_df, on = 'product_id')


In [20]:
price_history_df = price_history_df.drop(columns = ['product_id'])
price_history_df = price_history_df.rename(columns = {'id' : 'product_id'})
price_history_df.head()

,date,price,saleprice,product_id
0,02/08/2021,2.99,0.0,1
1,02/08/2021,1.99,0.0,2
2,02/08/2021,2.99,0.0,3
3,02/08/2021,1.99,0.0,4
4,02/08/2021,2.99,0.0,5


In [21]:
price_history_df.to_sql(name='Price_History', con=engine, if_exists='append', index=False)

In [22]:
pd.read_sql_query('select * from "Price_History"', con=engine).head()

,product_id,date,saleprice,price
0,1,2021-02-08,0.0,2.99
1,2,2021-02-08,0.0,1.99
2,3,2021-02-08,0.0,2.99
3,4,2021-02-08,0.0,1.99
4,5,2021-02-08,0.0,2.99
